In [2]:
def greet(name):
    print("Hello, " + name + "!")
    
greet("John")

Hello, John!


In [3]:
import ast

f = open("test_package.py", "r")

code = f.read()

tree = ast.parse(code)

In [4]:
f_np = open("packages/numpy-2.0.0rc1/numpy/core/fromnumeric.py", "r")

code_np = f_np.read()

tree_np = ast.parse(code_np)

As a single underscore in front of a function means, that it is meant to be private, those functions will not be included in the search index.

In [5]:
from _ast import *

for element in tree_np.body:
    if type(element) == FunctionDef and element.name[0] != "_":
        print(element.name)

In [6]:
import inspect
import importlib

m = importlib.import_module("numpy")


In [24]:
import pkgutil
import os

def get_all_functions_importlib(module_name):
    function_dict = {}
    class_dict = {}
    try:
        module = importlib.import_module(module_name)
    except ModuleNotFoundError as e:
        os.system("pip install {}".format(str(e).split("No module named ")[1][1:-1]))
        module = importlib.import_module(module_name)
    prefix = module.__name__ + "."
    for importer, sub_module_name, ispkg in pkgutil.iter_modules(module.__path__):
        if not ispkg and sub_module_name[0] != "_":
            try:
                sub_module = importlib.import_module(prefix + sub_module_name)
                source = inspect.getsource(sub_module)
                tree = ast.parse(source)
                function_list = []
                class_list = []
                for element in tree.body:
                    if type(element) == FunctionDef and element.name[0] != "_":
                        function_list.append(element.name)
                    elif type(element) == ClassDef and element.name[0] != "_":
                        class_list.append(element.name)
                if len(function_list) > 0:
                    function_dict[prefix + sub_module_name] = function_list
                if len(class_list) > 0:
                    class_dict[prefix + sub_module_name] = class_list
            except:
                pass
        elif ispkg:
            res_func_dict, res_class_dict = get_all_functions_importlib(prefix + sub_module_name)
            function_dict.update(res_func_dict)
            class_dict.update(res_func_dict)
    return function_dict, class_dict

module_name = "pandas"
os.system("pip install {}".format(module_name))
function_dict, class_dict = get_all_functions_importlib(module_name)


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [8]:
from os import listdir
from os.path import isfile, join, isdir

def get_all_functions_path(module_name, path):
    function_dict = {}
    class_dict = {}
    prefix = module_name + "."
    for element in sorted(listdir(path)):
        if isfile(join(path, element)):
            sub_module_name = element.split(".py")[0]
            source = open(join(path, element), "r").read()
            tree = ast.parse(source)
            function_list = []
            class_list = []
            for element in tree.body:
                if type(element) == FunctionDef and element.name[0] != "_":
                    function_list.append(element.name)
                elif type(element) == ClassDef and element.name[0] != "_":
                    class_list.append(element.name)
            if len(function_list) > 0:
                function_dict[prefix + sub_module_name] = function_list
            if len(class_list) > 0:
                class_dict[prefix + sub_module_name] = class_list
        elif isdir(join(path, element)):
            res_function_dict, res_class_dict = get_all_functions_path(prefix + element, join(path, element))
            function_dict.update(res_function_dict)
            class_dict.update(res_class_dict)
    return function_dict, class_dict
    
function_dict_path, class_dict_path = get_all_functions_path("numpy", "./packages/numpy-2.0.0rc1/numpy")